# IMPORTING LIBRARIES

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys
import json
import pickle
import glob 
import os
import warnings

In [71]:
import librosa

import tensorflow as tf
import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [42]:
warnings.filterwarnings(action="ignore")

np.random.seed(42)

%matplotlib inline

# READING DATASET

In [63]:
def load_data_SAVEE(
    SAVEE = "data/raw/SAVEE/"
):
    dir_list = sorted(os.listdir(SAVEE))

    emotion=[]
    path = []
    for i in dir_list:
        if i[-8:-6]=='_a':
            emotion.append('male_angry')
        elif i[-8:-6]=='_d':
            emotion.append('male_disgust')
        elif i[-8:-6]=='_f':
            emotion.append('male_fear')
        elif i[-8:-6]=='_h':
            emotion.append('male_happy')
        elif i[-8:-6]=='_n':
            emotion.append('male_neutral')
        elif i[-8:-6]=='sa':
            emotion.append('male_sad')
        elif i[-8:-6]=='su':
            emotion.append('male_surprise')
        else:
            emotion.append('male_error') 
        path.append(SAVEE + i)

    SAVEE_df = pd.DataFrame(emotion, columns = ['labels'])
    SAVEE_df['source'] = 'SAVEE'
    SAVEE_df = pd.concat([SAVEE_df, pd.DataFrame(path, columns = ['path'])], axis = 1)
    
    print(
        f"SAVEE_df:\n{SAVEE_df['labels'].value_counts()}",
        end="\n" + "-" * 100 + "\n"
    )

    return SAVEE_df

In [64]:
def load_data_RAVDESS(
    RAVDESS = "data/raw/RAVDESS/audio_speech_actors_01-24/"
):
    dir_list = sorted(os.listdir(RAVDESS))

    emotion = []
    gender = []
    path = []
    for i in dir_list:
        fname = os.listdir(RAVDESS + i)
        for f in fname:
            part = f.split('.')[0].split('-')
            emotion.append(int(part[2]))
            temp = int(part[6])
            if temp%2 == 0:
                temp = "female"
            else:
                temp = "male"
            gender.append(temp)
            path.append(RAVDESS + i + '/' + f)

        
    RAVDESS_df = pd.DataFrame(emotion)
    RAVDESS_df = RAVDESS_df.replace(
        {1:'neutral', 2:'neutral', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}
    )
    RAVDESS_df = pd.concat([pd.DataFrame(gender),RAVDESS_df],axis=1)
    RAVDESS_df.columns = ['gender','emotion']
    RAVDESS_df['labels'] =RAVDESS_df.gender + '_' + RAVDESS_df.emotion
    RAVDESS_df['source'] = 'RAVDESS'  
    RAVDESS_df = pd.concat([RAVDESS_df,pd.DataFrame(path, columns = ['path'])],axis=1)
    RAVDESS_df = RAVDESS_df.drop(['gender', 'emotion'], axis=1)
    
    print(
        f"RAVDESS_df:\n{RAVDESS_df['labels'].value_counts()}",
        end="\n" + "-" * 100 + "\n"
    )
    
    return RAVDESS_df

In [65]:
def load_data_TESS(
    TESS = "data/raw/TESS/TESS Toronto emotional speech set data/"
):
    dir_list=sorted(os.listdir(TESS))
    path = []
    emotion = []

    for i in dir_list:
        fname = os.listdir(TESS + i)
        for f in fname:
            if i == 'OAF_angry' or i == 'YAF_angry':
                emotion.append('female_angry')
            elif i == 'OAF_disgust' or i == 'YAF_disgust':
                emotion.append('female_disgust')
            elif i == 'OAF_Fear' or i == 'YAF_fear':
                emotion.append('female_fear')
            elif i == 'OAF_happy' or i == 'YAF_happy':
                emotion.append('female_happy')
            elif i == 'OAF_neutral' or i == 'YAF_neutral':
                emotion.append('female_neutral')                                
            elif i == 'OAF_Pleasant_surprise' or i == 'YAF_pleasant_surprised':
                emotion.append('female_surprise')               
            elif i == 'OAF_Sad' or i == 'YAF_sad':
                emotion.append('female_sad')
            else:
                emotion.append('Unknown')
            path.append(TESS + i + "/" + f)

    TESS_df = pd.DataFrame(emotion, columns = ['labels'])
    TESS_df['source'] = 'TESS'
    TESS_df = pd.concat([TESS_df,pd.DataFrame(path, columns = ['path'])],axis=1)
    
    print(
        f"TESS_df:\n{TESS_df['labels'].value_counts()}",
        end="\n" + "-" * 100 + "\n"
    )
    
    return TESS_df

In [69]:
def load_data_CREMA(
    CREMA = "data/raw/CREMA/"
):
    dir_list = sorted(os.listdir(CREMA))

    gender = []
    emotion = []
    path = []
    female = [
        1002,1003,1004,1006,1007,1008,1009,1010,1012,1013,1018,1020,
        1021,1024,1025,1028,1029,1030,1037,1043,1046,1047,1049,1052,
        1053,1054,1055,1056,1058,1060,1061,1063,1072,1073,1074,1075,
        1076,1078,1079,1082,1084,1089,1091
    ]

    for i in dir_list: 
        part = i.split('_')
        if int(part[0]) in female:
            temp = 'female'
        else:
            temp = 'male'
        gender.append(temp)
        if part[2] == 'SAD' and temp == 'male':
            emotion.append('male_sad')
        elif part[2] == 'ANG' and temp == 'male':
            emotion.append('male_angry')
        elif part[2] == 'DIS' and temp == 'male':
            emotion.append('male_disgust')
        elif part[2] == 'FEA' and temp == 'male':
            emotion.append('male_fear')
        elif part[2] == 'HAP' and temp == 'male':
            emotion.append('male_happy')
        elif part[2] == 'NEU' and temp == 'male':
            emotion.append('male_neutral')
        elif part[2] == 'SAD' and temp == 'female':
            emotion.append('female_sad')
        elif part[2] == 'ANG' and temp == 'female':
            emotion.append('female_angry')
        elif part[2] == 'DIS' and temp == 'female':
            emotion.append('female_disgust')
        elif part[2] == 'FEA' and temp == 'female':
            emotion.append('female_fear')
        elif part[2] == 'HAP' and temp == 'female':
            emotion.append('female_happy')
        elif part[2] == 'NEU' and temp == 'female':
            emotion.append('female_neutral')
        else:
            emotion.append('Unknown')
        path.append(CREMA + i)
    
    CREMA_df = pd.DataFrame(emotion, columns = ['labels'])
    CREMA_df['source'] = 'CREMA'
    CREMA_df = pd.concat([CREMA_df,pd.DataFrame(path, columns = ['path'])],axis=1)
    
    print(
        f"CREMA_df:\n{CREMA_df['labels'].value_counts()}",
        end="\n" + "-" * 100 + "\n"
    )
    
    return CREMA_df

In [67]:
def load_data():
    SAVEE_df = load_data_SAVEE()
    RAVDESS_df = load_data_RAVDESS()
    TESS_df = load_data_TESS()
    CREMA_df = load_data_CREMA()
        
    ref = pd.concat(
        [SAVEE_df, RAVDESS_df, TESS_df, CREMA_df],
        axis = 0
    )
    
    print(
        f"Total:\n{ref['labels'].value_counts()}",
        end="\n" + "-" *100 + "\n"
    )
    
    return ref

# FEATURE EXTRACTION

In [50]:
def feature_extraction(ref):
    df = pd.DataFrame(columns=['feature'])

    counter=0
    for index,path in enumerate(ref.path):
        X, sample_rate = librosa.load(path, res_type='kaiser_fast',duration=2.5,sr=44100,offset=0.5)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
        df.loc[counter] = [mfccs]
        counter=counter+1   
    
    ref.reset_index(inplace=True, drop=True)
    df = pd.concat([ref,pd.DataFrame(df['feature'].values.tolist())],axis=1)
    df=df.fillna(0)
    
    print(f"df.shape -> {df.shape}")
    
    return df

# TRAIN TEST SPLIT

In [11]:
def df_split(df):
    X_train, X_test, y_train, y_test = train_test_split(
        df.drop(
            ['path','labels','source'],axis=1
        ),
        df.labels,
        test_size=0.2,
        shuffle=True,
        random_state=42
    )
    
    return X_train, X_test, y_train, y_test

# NORMALIZATION

In [12]:
def normalization(X_train, X_test):
    mean = np.mean(X_train, axis=0)
    std = np.std(X_train, axis=0)

    X_train = (X_train - mean)/std
    X_test = (X_test - mean)/std
    return X_train, X_test

# MODELLING

### BASELINE TEST

In [90]:
def baseline_test(X_train, X_test, y_train, y_test):
    models = {
        "Logistic_regr": LogisticRegression(),
        "Decision_tree": DecisionTreeClassifier(),
        "Random_forest": RandomForestClassifier(
            max_features="log2",
            max_depth=10,
            max_leaf_nodes=100,
            min_samples_leaf=3,
            min_samples_split=20,
            n_estimators=22000,
            random_state=42
        )
    }
    for model in models:
        models[model].fit(X_train, y_train)
        
        print(
            f"{model} classification report:\n",
            classification_report(y_test, models[model].predict(X_test)),
            end="\n" + "-" * 100 + "\n"
        )

### Sequential model

In [91]:
def seq_model(X_train, X_test, y_train, y_test):
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)

    # one hot encode the target 
    lb = LabelEncoder()
    y_train = np_utils.to_categorical(lb.fit_transform(y_train))
    y_test = np_utils.to_categorical(lb.fit_transform(y_test))
    
    with open("data_description.txt", 'w') as f:
        f.write(f"{lb.classes_}")
    print(lb.classes_)
    
    X_train = np.expand_dims(X_train, axis=2)
    X_test = np.expand_dims(X_test, axis=2)
    
    
    # New model
    model = Sequential()
    model.add(Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1],1)))  # X_train.shape[1] = No. of Columns
    model.add(Activation('relu'))
    model.add(Conv1D(256, 8, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(MaxPooling1D(pool_size=(8)))
    model.add(Conv1D(128, 8, padding='same'))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 8, padding='same'))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 8, padding='same'))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 8, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(MaxPooling1D(pool_size=(8)))
    model.add(Conv1D(64, 8, padding='same'))
    model.add(Activation('relu'))
    model.add(Conv1D(64, 8, padding='same'))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(14))
    model.add(Activation('softmax'))

    opt = tf.keras.optimizers.RMSprop(lr=0.00001, decay=1e-6)
    
    print("train_ncols", X_train.shape[-1])
        
    model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
    model_history=model.fit(
        X_train,
        y_train,
        batch_size=16,
        epochs=80,
        verbose=1,
        validation_data=(X_test, y_test))
    
    return model

# MAIN

In [80]:
def main():
    ref = load_data()
    df = feature_extraction(ref)
    X_train, X_test, y_train, y_test = df_split(df)
    
    X_train, X_test = normalization(X_train, X_test)
    baseline_test(X_train, X_test, y_train, y_test)
    
    model = seq_model(X_train, X_test, y_train, y_test)

In [89]:
main()

SAVEE_df:
male_neutral     120
male_sad          60
male_happy        60
male_disgust      60
male_fear         60
male_surprise     60
male_angry        60
Name: labels, dtype: int64
----------------------------------------------------------------------------------------------------
RAVDESS_df:
female_neutral     144
male_neutral       144
male_sad            96
female_surprise     96
female_happy        96
male_fear           96
male_happy          96
female_sad          96
female_disgust      96
male_surprise       96
male_disgust        96
male_angry          96
female_angry        96
female_fear         96
Name: labels, dtype: int64
----------------------------------------------------------------------------------------------------
TESS_df:
female_surprise    400
female_happy       400
female_sad         400
female_fear        400
female_neutral     400
female_disgust     400
female_angry       400
Name: labels, dtype: int64
--------------------------------------------------------

Epoch 1/80
609/609 [==============================] - 72s 116ms/step - loss: 2.4212 - accuracy: 0.1778 - val_loss: 2.4091 - val_accuracy: 0.2314
Epoch 2/80
609/609 [==============================] - 82s 135ms/step - loss: 2.1800 - accuracy: 0.2635 - val_loss: 2.2173 - val_accuracy: 0.2737
Epoch 3/80
609/609 [==============================] - 83s 136ms/step - loss: 2.0665 - accuracy: 0.2944 - val_loss: 2.1285 - val_accuracy: 0.3206
Epoch 4/80
609/609 [==============================] - 79s 130ms/step - loss: 1.9871 - accuracy: 0.3224 - val_loss: 2.0688 - val_accuracy: 0.3333
Epoch 5/80
609/609 [==============================] - 74s 122ms/step - loss: 1.9230 - accuracy: 0.3441 - val_loss: 2.0176 - val_accuracy: 0.3498
Epoch 6/80
609/609 [==============================] - 74s 121ms/step - loss: 1.8629 - accuracy: 0.3649 - val_loss: 1.9823 - val_accuracy: 0.3584
Epoch 7/80
609/609 [==============================] - 74s 122ms/step - loss: 1.8321 - accuracy: 0.3690 - val_loss: 1.9400 - val_ac

609/609 [==============================] - 95s 156ms/step - loss: 1.2342 - accuracy: 0.5825 - val_loss: 1.6099 - val_accuracy: 0.4468
Epoch 58/80
609/609 [==============================] - 96s 157ms/step - loss: 1.2289 - accuracy: 0.5822 - val_loss: 1.6079 - val_accuracy: 0.4455
Epoch 59/80
609/609 [==============================] - 96s 158ms/step - loss: 1.2205 - accuracy: 0.5868 - val_loss: 1.6322 - val_accuracy: 0.4287
Epoch 60/80
609/609 [==============================] - 95s 157ms/step - loss: 1.2148 - accuracy: 0.5854 - val_loss: 1.6094 - val_accuracy: 0.4320
Epoch 61/80
609/609 [==============================] - 95s 157ms/step - loss: 1.2138 - accuracy: 0.5814 - val_loss: 1.6431 - val_accuracy: 0.4287
Epoch 62/80
609/609 [==============================] - 101s 166ms/step - loss: 1.2001 - accuracy: 0.5986 - val_loss: 1.6088 - val_accuracy: 0.4349
Epoch 63/80
609/609 [==============================] - 93s 153ms/step - loss: 1.1875 - accuracy: 0.5994 - val_loss: 1.5980 - val_accura